# 1. Data reading and preprocessing
 

## 1.1 imports

In [11]:
import json
import convokit
from convokit.politenessStrategies.politenessStrategies import PolitenessStrategies

import nltk
import pandas as pd
!python -m spacy download en
from convokit import Corpus, download
import pickle
import sklearn
#from convokit.classifier.classifier import Classifier
from convokit import TextParser
from nltk.tokenize.punkt import PunktSentenceTokenizer
from spacy.lang.en import English
deception = Corpus(filename=download("diplomacy-corpus"))
!pip3 install -U scikit-learn==0.19.2
import politeness
import joblib

Defaulting to user installation because normal site-packages is not writeable
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/cormak/.local/lib/python3.8/site-packages/en_core_web_sm -->
/home/cormak/.local/lib/python3.8/site-packages/spacy/data/en
You can now load the model via spacy.load('en')
Dataset already exists at /home/cormak/.convokit/downloads/diplomacy-corpus
Defaulting to user installation because normal site-packages is not writeable
Requirement already up-to-date: scikit-learn==0.19.2 in /home/cormak/.local/lib/python3.8/site-packages (0.19.2)


## 1.2 politeness

* I decided to take the annotated wiki corpus to train our politeness classifier as I could notfind a working pre trained model

In [35]:
parser = TextParser(verbosity=1000)
train_corpus = Corpus(filename=download('wiki-politeness-annotated'))
parser.transform(train_corpus)
ps = PolitenessStrategies()
ps.transform(train_corpus)
clf = Classifier(obj_type='utterance', pred_feats=['politeness_strategies'], 
                 labeller=lambda utt: utt.meta['Binary']==1)
clf.fit(train_corpus)

Dataset already exists at /home/cormak/.convokit/downloads/wiki-politeness-annotated
1000/4353 utterances processed
2000/4353 utterances processed
3000/4353 utterances processed
4000/4353 utterances processed
4353/4353 utterances processed
Initialized default classification model (standard scaled logistic regression).


* creation of features for politeness

In [5]:
politeness.model

AttributeError: module 'politeness' has no attribute 'model'

In [37]:

deception = parser.transform(deception)
deception = ps.transform(deception, markers=True)
summary = ps.summarize(deception)


1000/17289 utterances processed
2000/17289 utterances processed
3000/17289 utterances processed
4000/17289 utterances processed
5000/17289 utterances processed
6000/17289 utterances processed
7000/17289 utterances processed
8000/17289 utterances processed
9000/17289 utterances processed
10000/17289 utterances processed
11000/17289 utterances processed
12000/17289 utterances processed
13000/17289 utterances processed
14000/17289 utterances processed
15000/17289 utterances processed
16000/17289 utterances processed
17000/17289 utterances processed
17289/17289 utterances processed


In [38]:
politeness_scores = clf.transform(deception)

In [51]:
deception.random_utterance().meta

{'speaker_intention': 'Truth',
 'receiver_perception': 'Truth',
 'receiver': 'turkey-Game12',
 'absolute_message_index': 249,
 'relative_message_index': 74,
 'year': '1901',
 'game_score': '3',
 'game_score_delta': '-2',
 'deception_quadrant': 'Straightforward'}

In [36]:
import _pickle

MODEL_FILENAME='./models/politeness-svm.p'
clf = _pickle.load(open(MODEL_FILENAME, 'rb'), encoding='latin1', fix_imports=True)



from politeness.test_documents import TEST_DOCUMENTS
from politeness.features.vectorizer import PolitenessFeatureVectorizer
vectorizer = PolitenessFeatureVectorizer()

def score(request):
    """
    :param request - The request document to score
    :type request - dict with 'sentences' and 'parses' field
        sample (taken from test_documents.py)--
        {
            'sentences': [
                "Have you found the answer for your question?",
                "If yes would you please share it?"
            ],
            'parses': [
                ["csubj(found-3, Have-1)", "dobj(Have-1, you-2)",
                 "root(ROOT-0, found-3)", "det(answer-5, the-4)",
                 "dobj(found-3, answer-5)", "poss(question-8, your-7)",
                 "prep_for(found-3, question-8)"],
                ["prep_if(would-3, yes-2)", "root(ROOT-0, would-3)",
                 "nsubj(would-3, you-4)", "ccomp(would-3, please-5)",
                 "nsubj(it-7, share-6)", "xcomp(please-5, it-7)"]
            ]
        }
    returns class probabilities as a dict
        { 'polite': float, 'impolite': float }
    """
    # Vectorizer returns {feature-name: value} dict
    features = vectorizer.features(request)
    fv = [features[f] for f in sorted(features.keys())]
    # Single-row sparse matrix
    X = csr_matrix(np.asarray([fv]))
    probs = clf.predict_proba(X)
    # Massage return format
    probs = {"polite": probs[0][1], "impolite": probs[0][0]}
    return probs

from politeness.scripts.format_input import format_doc
doc_text = ""
with open(args[0], "r") as doc:
    doc_text = doc.read()
parsed_docs = format_doc(doc_text)
polite = []
impolite = []
for i, doc in enumerate(parsed_docs):
    probs = score(doc)
    polite.append(probs['polite'])
    impolite.append(probs['impolite'])
    print("\n====\nSentence " + str(i) + ":\n" + str(doc['sentences'][0]))
    print("\tP(polite) = %.3f" % probs['polite'])
    print("\tP(impolite) = %.3f" % probs['impolite'])

print("\n====\nDocument:")
print("\tP(polite) = %.3f" % np.mean(polite))
print("\tP(impolite) = %.3f" % np.mean(impolite))



ModuleNotFoundError: No module named 'features'